In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import  PorterStemmer
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [ ]:
from wordcloud import WordCloud

In [ ]:

from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline

ModuleNotFoundError: No module named 'wordcloud'

In [2]:
data = pd.read_csv('spam.csv', encoding="latin1")

data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [3]:
#Renaming columns and labeling data with 0's and 1's
data = data[['v1', 'v2']]

data = data.rename(columns={'v1':'label', 'v2':'message'})

data['label'].replace('ham', 0, inplace=True)
data['label'].replace('spam', 1, inplace=True)

data

C:\Users\aldin\AppData\Local\Temp\ipykernel_19624\435384640.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['label'].replace('ham', 0, inplace=True)
C:\Users\aldin\AppData\Local\Temp\ipykernel_19624\435384640.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [4]:
#Making data lowercase
data['message'] = data['message'].str.lower()

#Dividing data in train and test
train, test = train_test_split(data, test_size=0.25, random_state=65, shuffle=True, stratify=data['label'])

## Training

In [8]:
#To stemming the messages and removing stopwords
stemmer = PorterStemmer()

sw = stopwords.words('english')

def stem_text(msg):
    if pd.isna(msg):
        return None
    
    tokens = word_tokenize(msg)

    stemmed_tokens = [stemmer.stem(word) for word in tokens if word not in sw]

    return stemmed_tokens

In [12]:
#Dict to save the ocurrences of each word in spam and ham and all

spam_ocurr = defaultdict(int)
ham_ocurr = defaultdict(int)
all_ocurr = defaultdict(int)

#Filling the three dictionaries
total_words = 0
spam_words = 0
ham_words = 0

#Filling the dictionaries with the frecuency of each word according to the subgroup
for i, row in train.iterrows():
    words = stem_text(row['message'])
    if(row['label'] == 0):
        #Not spam
        ham_words += len(words)
        for w in words:
            ham_ocurr[w]+=1
            all_ocurr[w]+=1
    else:
        #print('In spam')
        #Spam
        spam_words += len(words)
        for w in words:
            spam_ocurr[w]+=1
            all_ocurr[w]+=1

total_words = spam_words + ham_words

In [18]:
print(f'Total words: {total_words}')
print(f'Spam words: {spam_words}')
print(f'Ham words: {ham_words}')

Total words: 52045
Spam words: 11661
Ham words: 40384


In [13]:
#Calculating  P(w|spam), P(w|~spam) and P(w) with additive smoothing

def prob_w_spam(word):
    return spam_ocurr[word]/spam_words
    
def prob_w_ham(word):
    return ham_ocurr[word]/ham_words

def prob_w(word):
    return all_ocurr[word]/total_words

### Common Words

In [ ]:
spam_words = ' '.join(list(mails[mails['label'] == 1]['message']))
spam_wc = WordCloud(width = 512,height = 512).generate(spam_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(spam_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

## Results

In [14]:
def calculate_metrics(true_pos, true_neg, false_pos, false_neg):
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    Fscore = 2 * precision * recall / (precision + recall)
    accuracy = (true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F-score: ", Fscore)
    print("Accuracy: ", accuracy)

In [15]:
#Comparing the model in training


def validate_performance(dataset):
    p_spam = dataset.loc[dataset['label'] == 1].shape[0]/ dataset.shape[0]
    p_ham = dataset.loc[dataset['label'] == 0].shape[0]/ dataset.shape[0]

    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0

    for _, row in dataset.iterrows():
        #print(phrase)
        phrase = row['message']
        phrase_stemmed = stem_text(phrase)
        prob_spam_p = p_spam
        prob_ham_p = p_ham
        for w in phrase_stemmed:
            #print(prob_w_spam(w), prob_w(w))
            
            if(spam_ocurr[w] != 0): #Validating that the word has appeared in the training set
                prob_spam_p *= (prob_w_spam(w))/(prob_w(w))
            
            if(ham_ocurr[w] != 0): #Validating that the word has appeared in the training set
                prob_ham_p *= (prob_w_ham(w))/(prob_w(w))

            pred_label = 0
            if(prob_spam_p>prob_ham_p): #Verifying if which probability is greater
                pred_label = 1

            if(pred_label == 1 and row['label'] == 1):
                true_pos += 1
            if(pred_label == 0 and row['label'] == 0):
                true_neg += 1
            if(pred_label == 1 and row['label'] == 0):
                false_pos += 1
            if(pred_label == 0 and row['label'] == 1):
                false_neg += 1

    calculate_metrics(true_pos, true_neg, false_pos, false_neg)

In [16]:
validate_performance(train)

Precision:  0.9438910719181212
Recall:  0.8857730897864677
F-score:  0.9139090426473191
Accuracy:  0.9626092804303967


In [17]:
validate_performance(test)

Precision:  0.9013806706114399
Recall:  0.8194159836065574
F-score:  0.8584462632496981
Accuracy:  0.9400363760372854
